In [431]:
from curses import flash
import os
from pickle import TRUE
from re import L
import requests
import validators
import gzip
import shutil
from urllib.parse import urlparse
import random

import sys
import math
import xarray as xa
import numpy as np
from rio_cogeo import cog_validate
import rioxarray

# Mapping
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [441]:
def open_file(input_path):
    return xa.open_dataset(input_file_path, engine="netcdf4", decode_coords='all', decode_times=False)

def copy_lat_lon_data(file, var_name1, var_name2, var_name3):
    lat = file[var_name1].data.copy()
    lon = file[var_name2].data.copy()
    data = file[var_name3].data[0].copy()
    return lat, lon, data

def make_new_xarray(lat, lon, data):
    file = xa.Dataset(
        data_vars={"flash_extent": (("lon", "lat"), data)},
        coords={"lon": lon, "lat": lat},
        attrs={"instrument_ID": "NALMA"}
    )
    return file

def generate_cog(file, variable_name):
    file = file[variable_name]
    file = file.transpose("lat","lon")
    file.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    file.rio.crs
    file.rio.set_crs('epsg:4326', inplace=True)
    name=f"S2A_20160724_135032_27XVB_B{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}.tif"
    cog_path=f"/home/asubedi/test_cog/cogs/{name}"
    file1.rio.to_raster(rf'{cog_path}', driver='COG')
    print(name)

In [442]:
input_file_path = "/home/asubedi/test_cog/NALMA_20230629_235000_600_10src_0.0109deg-dx_flash_extent.nc"
# input_file_path = "/home/asubedi/test_cog/WTLMA_20170531_235000_600_10src_0.0108deg-dx_flash_extent.nc"
variable_name = "flash_extent"

# input_file_path = '/home/asubedi/Desktop/data/raw-files/TRMM-LIS/lis_vhrfc_1998_2013_v01.nc'
# variable_name = "VHRFC_LIS_FRD"

In [443]:
file = open_file(input_file_path)
lat, lon, data = copy_lat_lon_data(file, 'latitude', 'longitude', variable_name)
file2 = make_new_xarray(lat, lon, data)

In [444]:
generate_cog(file2, variable_name)

ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this.